# Combining data and code

In [1]:
(ns joy.udp
  (:refer-clojure :exclude [get]))

nil

# UDP (Universal Design Pattern)

In [2]:
(defn beget [this proto]
    (assoc this ::prototype proto))

#'joy.udp/beget

In [3]:
(defn get [m k]
    (when m
        (if-let [[_ v] (find m k)]
            v
            (recur (::prototype m) k))))

#'joy.udp/get

In [4]:
(def put assoc)

#'joy.udp/put

## Basic usages

In [5]:
(def cat {:likes-dogs true, :ocd-bathing true})
(def morris (beget {:likes-9lives true} cat))
(def post-traumatic-morris (beget {:likes-dogs nil} morris))


#'joy.udp/post-traumatic-morris

In [6]:
(get cat :likes-dogs)

true

In [7]:
(get morris :likes-dogs)

true

In [8]:
(get post-traumatic-morris :likes-dogs)

nil

# Multimethods

In [9]:
(defmulti compiler :os)
(defmethod compiler ::unix [m] (get m :c-compiler))
(defmethod compiler ::osx  [m] (get m :llvm-compiler))

#multifn[compiler 0x71af3797]

In [10]:
(def clone (partial beget {}))
(def unix {:os ::unix, :c-compiler "cc", :home "/home", :dev "/dev"})
(def osx (-> (clone unix)
              (put :os ::osx)
              (put :llvm-compiler "clang")
              (put :home "/Users")))

#'joy.udp/osx

In [11]:
unix

{:os :joy.udp/unix, :c-compiler "cc", :home "/home", :dev "/dev"}

In [12]:
osx

{:joy.udp/prototype {:os :joy.udp/unix, :c-compiler "cc", :home "/home", :dev "/dev"}, :os :joy.udp/osx, :llvm-compiler "clang", :home "/Users"}

In [13]:
(compiler unix)

"cc"

In [14]:
(compiler osx)

"clang"

In [15]:
(defmulti home :os)
(defmethod home ::unix [m] (get m :home))

#multifn[home 0x56ce26f2]

In [16]:
(home unix)

"/home"

In [17]:
(home osx)

Execution error (IllegalArgumentException) at joy.udp/eval4144 (REPL:1).
No method in multimethod 'home' for dispatch value: :joy.udp/osx


class java.lang.IllegalArgumentException: 

In [18]:
; define unix is the parent of osx, 
; then we can call home on osx in which it will attempt to get from parenent
(derive ::osx ::unix)

nil

In [19]:
(home osx)

"/Users"

In [20]:
(isa? ::osx ::unix)

true

In [21]:
; what if osx has t parents?
(derive ::osx ::bsd)
(defmethod home ::bsd [m] "/home")

#multifn[home 0x56ce26f2]

In [22]:
(home osx)

Execution error (IllegalArgumentException) at joy.udp/eval4158 (REPL:1).
Multiple methods in multimethod 'home' match dispatch value: :joy.udp/osx -> :joy.udp/bsd and :joy.udp/unix, and neither is preferred


class java.lang.IllegalArgumentException: 

In [23]:
(prefer-method home ::unix ::bsd)
(home osx)

"/Users"

In [24]:
(remove-method home ::bsd)
(home osx)

"/Users"

## Arbitrary dispatch

In [25]:
(defmulti compile-cmd  (juxt :os compiler))

#'joy.udp/compile-cmd

In [26]:
(defmethod compile-cmd [::osx "clang"] [m] ; compiler call (juxt :os compiler) needd to return exactly ::osx "gcc"
  (str "/usr/bin/" (get m :c-compiler)))

(defmethod compile-cmd :default [m]
  (str "Unsure where to locate " (get m :c-compiler)))

#multifn[compile-cmd 0x1b94ca4a]

In [27]:
(compile-cmd osx)

"/usr/bin/cc"

In [28]:
(compile-cmd unix)

"Unsure where to locate cc"

# Records

In [29]:
(defrecord TreeNode [val l r])
; This creates a new Java class with a constructor that takes a value for each of the fields

joy.udp.TreeNode

In [30]:
(def tree (TreeNode. 5 nil nil))
tree

#joy.udp.TreeNode{:val 5, :l nil, :r nil}

In [31]:
(:val tree)

5

In [32]:
(defn xconj [t v] 
    (cond
        (nil? t)       (TreeNode. v nil nil)
        (< v (:val t)) (TreeNode. (:val t) (xconj (:l t) v) (:r t))
        :else          (TreeNode. (:val t) (:l t) (xconj (:r t) v))))

#'joy.udp/xconj

In [33]:
(def sample-tree (reduce xconj nil [3 5 2 4 6]))

#'joy.udp/sample-tree

# Protocols
think of it like interface in go lang

In [34]:
(defprotocol FIXO
      (fixo-push [fixo value])
      (fixo-pop [fixo])
      (fixo-peek [fixo]))

FIXO

In [35]:
(extend-type TreeNode
  FIXO
  (fixo-push [node value]
    (xconj node value)))

(fixo-push sample-tree 5/2)

#joy.udp.TreeNode{:val 3, :l #joy.udp.TreeNode{:val 2, :l nil, :r #joy.udp.TreeNode{:val 5/2, :l nil, :r nil}}, :r #joy.udp.TreeNode{:val 5, :l #joy.udp.TreeNode{:val 4, :l nil, :r nil}, :r #joy.udp.TreeNode{:val 6, :l nil, :r nil}}}

In [36]:
(extend-type clojure.lang.IPersistentVector
      FIXO
      (fixo-push [vector value]
        (conj vector value)))
(fixo-push [2 3 4 5 6] 5/2)

[2 3 4 5 6 5/2]

## Retify
retify is a more powerful way to do extend

In [37]:
; a stack-like FIXO that’s constrained to a certain fixed size
(defn fixed-fixo
  ([limit] (fixed-fixo limit []))
  ([limit vector]
    (reify FIXO
      (fixo-push [this value]
        (if (< (count vector) limit)
          (fixed-fixo limit (conj vector value))
          this))
      (fixo-peek [_]
        (peek vector))
      (fixo-pop [_]
        (pop vector)))))

#'joy.udp/fixed-fixo

## deftype

In [38]:
; For example if you want to implement the seq method for a type
; You can do it with record but you'll have to implement all the methods

(defrecord InfiniteConstant [i]
  clojure.lang.ISeq
  (seq [this]
    (lazy-seq (cons i (seq this)))))

; This is because record types are maps and implement everything maps should—seq along with assoc, dissoc, get, and so forth. 
; Because these are provided for you, you can’t implement them again yourself, and thus the preceding exception

Syntax error (ClassFormatError) compiling deftype* at (REPL:4:1).
Duplicate method name "seq" with signature "()Lclojure.lang.ISeq;" in class file joy/udp/InfiniteConstant


class clojure.lang.Compiler$CompilerException: 

In [39]:
; Clojure provides a lower-level deftype construct that’s similar to defrecord but doesn’t implement anything at all, 
; so implementing seq won’t conflict with anything
(deftype InfiniteConstant [i]
  clojure.lang.ISeq
  (seq [this]
    (lazy-seq (cons i (seq this)))))


joy.udp.InfiniteConstant

In [41]:
(take 3 (InfiniteConstant. 5))

(5 5 5)

# Chess

In [42]:
(defn build-move [& pieces]
    (apply hash-map pieces))

#'joy.udp/build-move

In [43]:
(build-move :from "e7" :to "e8" :promotion \Q)

{:from "e7", :promotion \Q, :to "e8"}

In [45]:
(defrecord Move [from to castle? promotion]
    Object
    (toString [this]
              (str "Move " (:from this)
                   " to " (:to this)
                   (if (:castle? this) " castle"
                       (if-let [p (:promotion this)]
                           (str " promote to " p) "")))))

joy.udp.Move

In [46]:
(str (Move. "e2" "e4" nil nil))

"Move e2 to e4"

In [48]:
(defn build-move [& {:keys [from to castle? promotion]}]
  {:pre [from to]}
  (Move. from to castle? promotion))

#'joy.udp/build-move

In [49]:
(str (build-move :from "e2" :to "e4"))

"Move e2 to e4"